BYNmasivofy2024*

In [169]:
# Importa las librerias a usar
import pandas as pd

K2

In [170]:
# Ubicacion de los archivos a usar
patch1 = '../input/Kilometros en Servicio MIO - 05-Mar-2024.xlsx'

In [171]:
# Se importa el archivo xlxs
kmsmio = pd.read_excel(patch1, sheet_name='Programacion')
# Se eliminan las filas y las columnas vacias
kmsmio = kmsmio.dropna(axis=0, how='all').dropna(axis=1, how='all')


In [172]:
kmsmio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20343 entries, 0 to 20342
Data columns (total 39 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Fecha                            20343 non-null  datetime64[ns]
 1   Día tipo                         20343 non-null  object        
 2   Designación de tarea vehículo    20343 non-null  object        
 3   Línea                            20343 non-null  object        
 4   Tipo de viaje corto              20343 non-null  object        
 5   Sentido                          13960 non-null  object        
 6   Tipo Kilómetros                  20343 non-null  object        
 7   Número de Vehículo               17622 non-null  object        
 8   Concesionario de Transporte      17572 non-null  object        
 9   Número de tarea vehículo         20343 non-null  object        
 10  Desde                            20343 non-null  object   

In [173]:
kmsmio.head()

,Fecha,Día tipo,Designación de tarea vehículo,Línea,Tipo de viaje corto,Sentido,Tipo Kilómetros,Número de Vehículo,Concesionario de Transporte,Número de tarea vehículo,...,VIAJE,VEHICULO SGCO,Kms en Servicio (MTS PLANEADOS),PARADAS PLANEADAS,PARADAS REALIZADAS SGCO,% PARADAS REALIZADAS SGCO,Kms en Servicio (MTS SGCO),FUENTE SGCO,HORA REAL SALIDA,HORA REAL LLEGADA
0,2024-03-05,DHABIL,1,A14A,Viaje de salida,NaN,Kms en Vacio,32013,ETM,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-03-05,DHABIL,1,A14A,Viaje comercial,Ida,Kms en Servicio,32013,ETM,1,...,1.0,32013.0,5311.0,6.0,6.0,1.0,5311.0,Control,05:01:50,05:15:49
2,2024-03-05,DHABIL,1,A14A,Viaje comercial,Vuelta,Kms en Servicio,32013,ETM,1,...,2.0,32013.0,4419.0,11.0,11.0,1.0,4419.0,Control,05:16:44,05:31:45
3,2024-03-05,DHABIL,1,A14A,Viaje comercial,Ida,Kms en Servicio,32013,ETM,1,...,9.0,32013.0,5311.0,6.0,6.0,1.0,5311.0,Control,05:32:33,05:47:16
4,2024-03-05,DHABIL,1,A14A,Viaje comercial,Vuelta,Kms en Servicio,32013,ETM,1,...,10.0,32013.0,4419.0,11.0,11.0,1.0,4419.0,Control,05:49:27,06:01:47


In [174]:
# Filtra el DF por Kms en Servicio
kmsmio = kmsmio[kmsmio['Tipo Kilómetros']== 'Kms en Servicio']
kmsmio = kmsmio[kmsmio['Kilometros Programados Plan'].notna()]

In [175]:
# Cambiamos la hora 24 por 0
kmsmio['Desde'] = kmsmio['Desde'].replace('24', '0')
kmsmio = kmsmio[kmsmio['Desde'] != '0']


In [176]:
# Se crea una nueva columna 'id'
kmsmio['id'] = kmsmio.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['Desde']), str(row['hasta'])]), axis=1)


In [177]:
kmsmio.shape

(18645, 40)

In [178]:
kmsmio['id'].value_counts()

id
A14A-1-05:00:00-05:12:10                              1
T51-562-09:33:30-10:30:24                             1
P51A-559-13:58:14-13:59:14                            1
P51A-559-13:01:28-13:57:59                            1
P51A-559-12:00:39-13:00:58                            1
                                                     ..
P21C-235-17:20:17-18:22:37                            1
E21-234-07:48:36-08:47:15                             1
E21-234-07:44:14-07:45:14                             1
E21-234-06:44:00-07:43:59                             1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

Reporte finalizado

In [179]:
patch2 =  '../input/REPORTE FINALIZADO 05_MARZ_2024.xlsx'

In [180]:
# Se importa el archivo xlxs
repofi = pd.read_excel(patch2, sheet_name='Programacion')
# Se eliminan las filas y las columnas vacias
repofi = repofi.dropna(axis=0, how='all').dropna(axis=1, how='all')

In [181]:
# Filtra el DF por Kms en Servicio
repofi = repofi[repofi['Tipo de kilometros']== 'Kms en Servicio']


In [182]:
# Se crea una nueva columna 'id'
repofi['id'] = repofi.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                  str(row['desde']),str(row['hasta'])]), axis=1)

In [183]:
repofi['id'].value_counts()

id
A14A-1-05:00:00-05:12:10                              1
T51-562-09:33:30-10:30:24                             1
P51A-559-13:58:14-13:59:14                            1
P51A-559-13:01:28-13:57:59                            1
P51A-559-12:00:39-13:00:58                            1
                                                     ..
P21C-235-17:20:17-18:22:37                            1
E21-234-07:48:36-08:47:15                             1
E21-234-07:44:14-07:45:14                             1
E21-234-06:44:00-07:43:59                             1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

In [184]:
repofi.shape

(18645, 32)

Merge

In [185]:
merge = pd.merge(kmsmio, repofi, on='id', how='outer')

In [188]:
merge.shape

(18645, 71)

In [187]:
merge.head()

,Fecha_x,Día tipo_x,Designación de tarea vehículo_x,Línea_x,Tipo de viaje corto_x,Sentido_x,Tipo Kilómetros,Número de Vehículo,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Tipo de vehículo del viaje.1,Incumplidos_y,Nro de caso,Tipo,Lugar,Jerarquia,Vehículo,Kms Planeados,Kms Perdidos,Kms cancelados
0,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,13,...,DUA,2.0,I20051442,Incumplimiento del COT,SANBO2,Incumplimiento del COT::Flota::Combustible,54004.0,4704,4704.0,NaN
1,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,13,...,DUA,2.0,NaN,NaN,NaN,NaN,NaN,4704,4704.0,NaN
2,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,54013,BYNCPC,13,...,DUA,NaN,20022922,Retoma,SANBO2,Retoma,54013.0,4704,NaN,NaN
3,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,54013,BYNCPC,13,...,DUA,NaN,2374293,Incidente,SANPAS6,Incidente::Por Flota::Con Protocolo Tecnologic...,54013.0,4704,NaN,NaN
4,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,54013,BYNCPC,13,...,DUA,NaN,NaN,NaN,NaN,NaN,NaN,4704,NaN,NaN


In [145]:
print(merge.columns)

Index(['Fecha_x', 'Día tipo_x', 'Designación de tarea vehículo_x', 'Línea_x',
       'Tipo de viaje corto_x', 'Sentido_x', 'Tipo Kilómetros',
       'Número de Vehículo', 'Concesionario de Transporte_x',
       'Número de tarea vehículo_x', 'Desde', 'hasta_x', 'Duración_x',
       'Punto de inicio_x', 'Punto de término_x', 'Largo_x',
       'Número de viaje_x', 'Tipo de vehículo del viaje_x',
       'Secuencia de arcos_x', 'Descripción Novedad_x',
       'Concesionario Programado', 'Tipo de vehículo Programado',
       'Incumplidos_x', 'Kilometros Programados-Desvios',
       'Kilometros Programados Plan', 'Clase de viaje', 'a evaluar', 'TAREA',
       'LINEA', 'VIAJE', 'VEHICULO SGCO', 'Kms en Servicio (MTS PLANEADOS)',
       'PARADAS PLANEADAS', 'PARADAS REALIZADAS SGCO',
       '% PARADAS REALIZADAS SGCO', 'Kms en Servicio (MTS SGCO)',
       'FUENTE SGCO', 'HORA REAL SALIDA', 'HORA REAL LLEGADA', 'id', 'Fecha_y',
       'Día tipo_y', 'Designación de tarea vehículo_y', 'Línea_y',
 

In [146]:
merge.shape

(18645, 71)

In [152]:
filtro = merge[merge['Largo_x'] != merge['Largo_y']]

In [189]:
filtro[filtro['Número de tarea vehículo_x']== 6]

,Fecha_x,Día tipo_x,Designación de tarea vehículo_x,Línea_x,Tipo de viaje corto_x,Sentido_x,Tipo Kilómetros,Número de Vehículo,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Tipo de vehículo del viaje.1,Incumplidos_y,Nro de caso,Tipo,Lugar,Jerarquia,Vehículo,Kms Planeados,Kms Perdidos,Kms cancelados
10800,2024-03-05,DHABIL,6,P21A,Viaje comercial,Vuelta,Kms en Servicio,22125,BYN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10801,2024-03-05,DHABIL,6,P21A,Viaje comercial,Ida,Kms en Servicio,22125,BYN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10802,2024-03-05,DHABIL,6,P21A,Viaje comercial,Vuelta,Kms en Servicio,22125,BYN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10803,2024-03-05,DHABIL,6,P21A,Viaje comercial,Ida,Kms en Servicio,22125,BYN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10804,2024-03-05,DHABIL,6,P21A,Viaje comercial,Vuelta,Kms en Servicio,22125,BYN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10805,2024-03-05,DHABIL,6,P21A,Viaje comercial,Ida,Kms en Servicio,22125,BYN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10806,2024-03-05,DHABIL,6,P21A,Viaje comercial,Vuelta,Kms en Servicio,22125,BYN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10807,2024-03-05,DHABIL,6,P21A,Viaje comercial,Ida,Kms en Servicio,22125,BYN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10808,2024-03-05,DHABIL,6,P21A,Viaje comercial,Vuelta,Kms en Servicio,CAMBIO,NaN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10809,2024-03-05,DHABIL,6,P21A,Viaje comercial,Ida,Kms en Servicio,CAMBIO,NaN,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
merge.to_excel('../output/prueba_merge.xlsx', index=False)